# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize



from sklearn.metrics import mean_absolute_error, mean_squared_error

from scipy.stats import skew, moment, kurtosis

from statsmodels.tsa.arima.model import ARIMA
#from statsmodels.tsa.arima_model import ARIMA
#from statsmodels.tsa.statespace.sarimax import SARIMAX

import os
#import warnings
#warnings.simplefilter(action = "module", category=FutureWarning)


%matplotlib inline

# Creating DataFrames

## Crypto DataFrame

In [2]:
folder_path = os.path.abspath("C:\\Users\\Corey Feld\\Documents\\Year2Semester1\\FIN611\\ResearchProject\\CryptoData")
files = os.listdir(folder_path)

df_crypto = pd.DataFrame()

for file in files:
    df_temp = pd.read_csv("C:\\Users\\Corey Feld\\Documents\\Year2Semester1\\FIN611\\ResearchProject\\CryptoData\\" + file)
    df_temp["Date"] = pd.to_datetime(df_temp["Date"]).dt.date
    df_temp["PercentChange"] = df_temp["Close"].pct_change() * 100
    df_temp["StockName"] = str(file[:-4])
    df_temp = df_temp[["Date", "PercentChange", "StockName"]]
    df_crypto = pd.concat([df_crypto, df_temp])
df_crypto = pd.pivot_table(data = df_crypto, values = "PercentChange", index = "Date", columns = "StockName")
df_crypto.head()

StockName,Basic Attention Token,Bitcoin Gold,Dash,Decentraland,Decred,Enjin Coin,Gnosis,Holo,KuCoin Token,Loopring,...,coin_ChainLink,coin_Dogecoin,coin_EOS,coin_Ethereum,coin_Iota,coin_Litecoin,coin_Monero,coin_Stellar,coin_Tron,coin_XRP
Date,,,,,,,,,,,,,,,,,,,,,
2013-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.993886,NaN,NaN,NaN,NaN
2013-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-11.532211,NaN,NaN,NaN,NaN
2013-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-11.287262,NaN,NaN,NaN,NaN
2013-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-9.699642,NaN,NaN,NaN,NaN
2013-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14.276614,NaN,NaN,NaN,NaN


## SP500 DataFrame

In [3]:
folder_path = os.path.abspath("C:\\Users\\Corey Feld\\Documents\\Year2Semester1\\FIN611\\ResearchProject\\SP500Data")
files = os.listdir(folder_path)

df_sp500 = pd.DataFrame()

for file in files:
    df_temp = pd.read_csv("C:\\Users\\Corey Feld\\Documents\\Year2Semester1\\FIN611\\ResearchProject\\SP500Data\\" + file)
    df_temp["Date"] = pd.to_datetime(df_temp["Date"]).dt.date
    df_temp["PercentChange"] = df_temp["Close"].pct_change() * 100
    df_temp["StockName"] = str(file[:-4])
    df_temp = df_temp[["Date", "PercentChange", "StockName"]]
    df_sp500 = pd.concat([df_sp500, df_temp])
df_sp500 = pd.pivot_table(data = df_sp500, values = "PercentChange", index = "Date", columns = "StockName")
df_sp500.head()

StockName,AAPL,ABBV,ACN,AMD,AMZN,AVGO,BAC,BRK-B,COST,CSCO,...,PFE,PG,TMO,TSLA,UNH,V,WMT,XOM,^IXIC,^RUT
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,-2.505933,-0.416598,0.520474,-0.722022,-0.502363,-3.345512,0.000000,0.382409,0.244453,-0.454370,...,0.719874,NaN,-0.172977,0.008950,0.188907,0.752972,2.375527,0.852058,-0.237802,0.164163
2016-01-06,-1.956970,0.017434,-0.195386,-8.727273,-0.179873,-3.075924,-2.130250,0.060954,-0.925402,-1.065048,...,-1.771283,NaN,-0.765286,-1.964817,-1.019885,-1.311132,1.001273,-0.832056,-1.138121,-1.447169
2016-01-07,-4.220457,-0.296274,-2.936570,-9.163347,-3.905796,-3.181027,-3.606965,-1.408670,-2.290937,-2.306805,...,-0.664350,NaN,-2.284466,-1.547662,-2.943980,-1.966250,2.328875,-1.600617,-3.025990,-2.723032
2016-01-08,0.528776,-2.726791,-0.968139,-6.140351,-0.146389,-0.658664,-1.935484,-0.888163,-1.750424,-2.479335,...,-1.273887,NaN,-0.111677,-2.156272,-1.721824,-1.233235,-2.291247,-2.020203,-0.976671,-1.725579
2016-01-11,1.619224,-3.180595,1.048886,9.345794,1.760969,-0.124808,0.723684,0.023376,1.715864,1.977397,...,0.225808,NaN,0.536673,-1.492891,-0.526509,1.427005,1.070192,-1.338867,-0.121449,-0.411004


## Commodity DataFrame

In [4]:
folder_path = os.path.abspath("C:\\Users\\Corey Feld\\Documents\\Year2Semester1\\FIN611\\ResearchProject\\CommodityData")
files = os.listdir(folder_path)

df_comm = pd.DataFrame()

for file in files:
    df_temp = pd.read_csv("C:\\Users\Corey Feld\Documents\Year2Semester1\FIN611\ResearchProject\CommodityData\\" + file)
    df_temp["Date"] = pd.to_datetime(df_temp["Date"]).dt.date
    df_temp["PercentChange"] = df_temp["Change %"].str[:-1].astype("float64")
    df_temp.drop(["Change %"], axis = 1, inplace = True)
    df_temp["StockName"] = str(file[:-28])
    df_temp = df_temp[["Date", "PercentChange", "StockName"]]
    df_comm = pd.concat([df_comm, df_temp])

#df_comm.columns = df_comm.columns.str.strip("'")
df_comm = pd.pivot_table(data = df_comm, values = "PercentChange", index = "Date", columns = "StockName")
df_comm.head()

StockName,Brent Oil,Canola,Class III Milk,Cobalt,Copper,Crude Oil WTI,Feeder Cattle,Gasoline RBOB,Gold,Heating Oil,...,Silver,Tin,US Cocoa,US Coffee C,US Corn,US Cotton #2,US Soybeans,US Sugar #11,US Wheat,Zinc
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.16,-0.57,0.00,-1.28,-2.60,-0.31,0.43,-0.04,1.41,2.33,...,0.28,-0.55,-2.59,-2.17,-2.02,-1.15,-0.77,-1.77,-2.50,-2.55
2016-01-05,-2.15,0.51,-0.44,0.00,0.77,-0.92,0.10,-2.63,0.30,-0.10,...,0.94,-1.87,-2.02,-0.71,0.43,0.17,0.38,-2.67,0.65,0.42
2016-01-06,-6.01,0.38,0.15,0.00,-0.36,-2.13,0.32,-7.55,1.25,-3.96,...,0.04,-2.61,-2.52,-2.40,0.07,-0.89,0.92,-1.03,0.33,-2.07
2016-01-07,-1.40,-1.69,0.37,0.00,-3.16,-1.73,-2.67,-1.36,1.46,-1.40,...,2.63,-0.69,-0.60,-1.23,-0.07,-1.25,0.14,2.29,1.24,-3.34
2016-01-08,-0.59,0.70,0.81,0.00,0.00,-0.15,-2.68,-1.60,-0.90,-1.27,...,-2.97,0.15,1.72,0.50,1.13,0.20,0.29,-1.97,2.13,0.94


# ARIMA Function

In [6]:
def arima(df):
    
    global mse_test_dict
    mse_test_dict = {}

    global rmse_test_dict
    rmse_test_dict = {}

    global mae_test_dict
    mae_test_dict = {}

    for col in df:
        asset = df[col]
        #print(str(asset))
        asset.dropna(inplace = True)
        asset = asset.to_numpy()
        asset = winsorize(asset, limits = [0.1, 0.1])
        asset = pd.Series(asset)
        train_size = int(len(asset) * 0.9)
        train_data = asset.iloc[:train_size]
        test_data = list(asset.iloc[train_size:len(asset)])
        history = [x for x in train_data]
        predictions = []
        
        for l in range(len(test_data)):
            model = ARIMA(history, order = (1, 0, 1))
            model_fit = model.fit()
            output = model_fit.forecast()
            y_hat = output[0]
            predictions.append(y_hat)
            obs = test_data[l]
            history.append(obs)
            #print("predicted = %f, expected = %f" % (y_hat, obs))
            
        mse_test = mean_squared_error(test_data, predictions)
        rmse_test = np.sqrt(mse_test)
        mae_test = mean_absolute_error(test_data, predictions)
        
        mse_test_dict[col] = mse_test
        rmse_test_dict[col] = rmse_test
        mae_test_dict[col] = mae_test
        
        #print("predicted")
        
    mse_test_dict = sorted(mse_test_dict.items(), key = lambda x:x[1])
    rmse_test_dict = sorted(rmse_test_dict.items(), key = lambda x:x[1])
    mae_test_dict = sorted(mae_test_dict.items(), key = lambda x:x[1])

# ARIMA on Crypto DataFrame

In [7]:
arima(df_crypto)

arima_crypto_mse_dict = mse_test_dict
arima_crypto_rmse_dict = rmse_test_dict
arima_crypto_mae_dict = mae_test_dict
print(arima_crypto_mse_dict)
print("-----------------------------------------------------------------------")
print(arima_crypto_rmse_dict)
print("-----------------------------------------------------------------------")
print(arima_crypto_mae_dict)

C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarni

[('coin_Bitcoin', 7.369539670863075), ('KuCoin Token', 10.890228666214355), ('Decred', 13.195893869401567), ('Bitcoin Gold', 14.22805085372595), ('Gnosis', 14.401228629279643), ('coin_Litecoin', 15.281474192499171), ('NEM', 15.31269003847788), ('Dash', 15.49762759679811), ('Neo', 15.89709652847474), ('Qtum', 17.58738141837816), ('coin_XRP', 17.73621533760283), ('coin_Monero', 18.116453590518244), ('coin_Dogecoin', 18.226899911781054), ('Basic Attention Token', 18.71281249213075), ('coin_Ethereum', 19.052500771461524), ('Tezos', 19.484021311448807), ('Zcash', 19.752561240875238), ('coin_Stellar', 20.31731233332717), ('Holo', 20.523427675259367), ('Decentraland', 21.22489075189725), ('Waves', 21.509900556461957), ('Enjin Coin', 22.865921120388638), ('Zilliqa', 23.776975310078523), ('Theta Network', 24.16694266841033), ('coin_Cardano', 24.59521944573045), ('coin_BinanceCoin', 25.057152318582478), ('Loopring', 25.184827151602295), ('coin_Tron', 25.436705565583946), ('coin_EOS', 27.12104229

# ARIMA on SP500 DataFrame

In [8]:
arima(df_sp500)

arima_sp500_mse_dict = mse_test_dict
arima_sp500_rmse_dict = rmse_test_dict
arima_sp500_mae_dict = mae_test_dict
print(arima_sp500_mse_dict)
print("-----------------------------------------------------------------------")
print(arima_sp500_rmse_dict)
print("-----------------------------------------------------------------------")
print(arima_sp500_mae_dict)

C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarni

[('PEP', 0.6548889575189777), ('JNJ', 0.663999121848883), ('KO', 0.6733496453301013), ('MCD', 0.7674931098891096), ('MRK', 0.8096912587310349), ('BRK-B', 0.8589804232185589), ('WMT', 0.8690120058318069), ('PG', 0.9026656797478709), ('ABBV', 1.1458005443735348), ('COST', 1.2009083599515777), ('UNH', 1.2414585795808064), ('PFE', 1.3464672501324242), ('ACN', 1.3565544758761865), ('HD', 1.3705015218427603), ('CSCO', 1.423469265378476), ('V', 1.4801943323416413), ('^IXIC', 1.4961457527813826), ('^RUT', 1.546342687948852), ('LLY', 1.5598920860172458), ('TMO', 1.695990097459796), ('MA', 1.7782917375919014), ('JPM', 1.8034884127568944), ('DIS', 1.8394294543947582), ('MSFT', 1.9650062626250513), ('CVX', 2.116688047207226), ('GOOG', 2.1363185433586453), ('GOOGL', 2.1591260058342283), ('BAC', 2.230985812361441), ('AAPL', 2.2930777469361114), ('XOM', 2.3983722290863843), ('AVGO', 2.7868383629119755), ('AMZN', 2.9899904459171975), ('META', 3.2446992346911667), ('NVDA', 6.948597677607786), ('TSLA', 

# ARIMA on Commodity DataFrame

In [9]:
arima(df_comm)

arima_comm_mse_dict = mse_test_dict
arima_comm_rmse_dict = rmse_test_dict
arima_comm_mae_dict = mae_test_dict
print(arima_comm_mse_dict)
print("-----------------------------------------------------------------------")
print(arima_comm_rmse_dict)
print("-----------------------------------------------------------------------")
print(arima_comm_mae_dict)

C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Anaconda\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Anaconda\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoreg

[('Class III Milk', 0.09069743177492476), ('Cobalt', 0.1319747504951177), ('Gold', 0.4360103808297496), ('Live Cattle', 0.4551804160247592), ('Feeder Cattle', 0.7313753108709476), ('Rough Rice', 0.962200840448276), ('Canola', 1.0099008049977), ('Tin', 1.1381789442464545), ('Rubber RSS3', 1.1671799827882197), ('Copper', 1.254450816897082), ('US Soybeans', 1.256053450615211), ('Lead', 1.3668059508668147), ('US Sugar #11', 1.495658812851008), ('Platinum', 1.529079094327361), ('US Corn', 1.5663764246738696), ('Silver', 1.6460180683239685), ('US Cocoa', 1.6463213315714798), ('Lean Hogs', 1.7920570887237794), ('Zinc', 1.8593879447603698), ('US Cotton #2', 1.8724743687671488), ('Crude Oil WTI', 2.3944678456589052), ('Palladium', 2.7312604295950065), ('US Coffee C', 2.7531247840329502), ('Nickel', 2.881394610102578), ('Orange Juice', 3.019377065868044), ('US Wheat', 3.0882863625960932), ('Oats', 3.2723900706671096), ('Heating Oil', 3.649894617463918), ('Brent Oil', 3.7786791320203443), ('Londo

# Stored Variables

In [10]:
%store arima_crypto_mse_dict
%store arima_crypto_rmse_dict
%store arima_crypto_mae_dict

%store arima_sp500_mse_dict
%store arima_sp500_rmse_dict
%store arima_sp500_mae_dict

%store arima_comm_mse_dict
%store arima_comm_rmse_dict
%store arima_comm_mae_dict

Stored 'arima_crypto_mse_dict' (list)
Stored 'arima_crypto_rmse_dict' (list)
Stored 'arima_crypto_mae_dict' (list)
Stored 'arima_sp500_mse_dict' (list)
Stored 'arima_sp500_rmse_dict' (list)
Stored 'arima_sp500_mae_dict' (list)
Stored 'arima_comm_mse_dict' (list)
Stored 'arima_comm_rmse_dict' (list)
Stored 'arima_comm_mae_dict' (list)
